# Age Classification
In this project,we are implementing age classification using MobileNet.

## Data
We will be using the IMDB Wiki dataset (https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/) for this project. We have already cleaned up the dataset to remove the non-face images as well as images with erroneous ages (age > 100 or age < 0).

In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

Using TensorFlow backend.


# Model
We have used the output layer of MobileNet as the base model. Then we have added three fully connected layers and a dropout layer. Since we have ages between 1 to 99, last layer will have 99 nodes. We have used softmax activation function to classify the age between the 99 output classes.

In [2]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dropout(0.5)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(99,activation='softmax')(x) #final layer with softmax activation

/home/ubuntu/anaconda3/envs/tf_env/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


# Network
We are freezing first 20 layers in the network and we will train the subsequent layers.

In [3]:
model=Model(inputs=base_model.input,outputs=preds)

#now a model has been created based on our architecture
# set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

Apart from measuring the classification loss, we will also measure the mean absolute error in prediction of the age.

In [4]:
def age_mae(y_true, y_pred):
    true_age = K.sum(y_true * K.arange(1, 100, dtype="float32"), axis=-1)
    pred_age = K.sum(y_pred * K.arange(1, 100, dtype="float32"), axis=-1)
    mae = K.mean(K.abs(true_age - pred_age))
    return mae

# Data Generator
Let's create a data generator that loads the image, class pairs from our trainig data directory. We will also set up the loss function and the optimizer for our model.

In [5]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('imdb_classes/train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=[age_mae])

# checkpoint
filepath="weights-improvement-stage2-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
step_size_train=train_generator.n//train_generator.batch_size

Found 131003 images belonging to 99 classes.


Let's also add a data generator for our validation data and run the model.

In [8]:
val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

val_generator=val_datagen.flow_from_directory('imdb_classes/val/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)
step_size_val=val_generator.n//val_generator.batch_size

model.fit_generator(generator=train_generator,
                    validation_data=val_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=step_size_val,
                    epochs=10)

Found 43624 images belonging to 99 classes.
Epoch 1/10
4093/4093 [==============================] - 1448s 354ms/step - loss: 3.1082 - age_mae: 5.4260 - val_loss: 3.5256 - val_age_mae: 7.1250
Epoch 2/10
4093/4093 [==============================] - 1439s 352ms/step - loss: 3.0437 - age_mae: 5.1479 - val_loss: 3.5054 - val_age_mae: 7.0546
Epoch 3/10
4093/4093 [==============================] - 1423s 348ms/step - loss: 2.9865 - age_mae: 4.9535 - val_loss: 3.5760 - val_age_mae: 7.3682
Epoch 4/10
4093/4093 [==============================] - 1452s 355ms/step - loss: 2.9242 - age_mae: 4.7499 - val_loss: 3.6316 - val_age_mae: 7.6298
Epoch 5/10
4093/4093 [==============================] - 1454s 355ms/step - loss: 2.8745 - age_mae: 4.5896 - val_loss: 3.6783 - val_age_mae: 7.3103
Epoch 6/10
4093/4093 [==============================] - 1429s 349ms/step - loss: 2.8262 - age_mae: 4.4561 - val_loss: 3.6986 - val_age_mae: 7.2361
Epoch 7/10
4093/4093 [==============================] - 1417s 346ms/step -

# Result
So we can see that we are able to achieve a better mean absolute error (7.3 years) with this simple neural network than what we could get with our SVM using LBP features (mae = 10 years).